In [2]:
# Import bibliotek
import numpy as np
import matplotlib.pyplot as plt
import os
from copy import *

%matplotlib qt

In [3]:
# Warunki początkowe oraz jednostki
au = 149597870700   #m
G = 6.67411e-11     #
x0 = 0              #m
y0 = 0.586*au       #m
vx0 = 54600         #m/s
vy0 = 0             #m/s
msol = 1.989e30     #kg

# Zmienne globalne
folderpath = 'z2_pictures'

In [5]:
# Funkcje pomocnicze
def radius(x, y):
    return np.sqrt(x**2 + y**2)

def getAccX(x, y):
    return -G*msol*x/(radius(x, y)**3)

def getAccY(x, y):
    return -G*msol*y/(radius(x, y)**3)

def getRK4k(xPrev, yPrev, vxPrev, vyPrev, dt):

    k = {   'x':np.empty(4),
            'y':np.empty(4),
            'vx':np.empty(4),
            'vy':np.empty(4)}

    k['x'][0] = vxPrev
    k['y'][0] = vyPrev
    k['vx'][0] = getAccX(xPrev, yPrev)
    k['vy'][0] = getAccY(xPrev, yPrev)

    k['x'][1] = vxPrev + k['vx'][0]*dt/2
    k['y'][1] = vyPrev + k['vy'][0]*dt/2
    k['vx'][1] = getAccX(xPrev+k['x'][0]*dt/2, yPrev+k['y'][0]*dt/2)
    k['vy'][1] = getAccY(xPrev+k['x'][0]*dt/2, yPrev+k['y'][0]*dt/2)

    k['x'][2] = vxPrev + k['vx'][1]*dt/2
    k['y'][2] = vyPrev + k['vy'][1]*dt/2
    k['vx'][2] = getAccX(xPrev+k['x'][1]*dt/2, yPrev+k['y'][1]*dt/2)
    k['vy'][2] = getAccY(xPrev+k['x'][1]*dt/2, yPrev+k['y'][1]*dt/2)

    k['x'][3] = vxPrev + k['vx'][2]*dt
    k['y'][3] = vyPrev + k['vy'][2]*dt
    k['vx'][3] = getAccX(xPrev+k['x'][2]*dt, yPrev+k['y'][2]*dt)
    k['vy'][3] = getAccY(xPrev+k['x'][2]*dt, yPrev+k['y'][2]*dt)
    
    return k

def estimateError(u1, u2, method):
    if method == 'euler':
        return abs(u1-u2)
    elif method == 'rk4':
        return abs((u1-u2)/15)
    else:
        pass

In [7]:
# Warunki symulacji metodą jawnego schematu Eulera
dt = 15*60                                      #15 minut
tmax = int(3.75*76*365*24*60*60)                #3 okresy
time_interval = int(tmax/dt)
t = np.arange(0, tmax, dt, dtype = float)
x = np.empty(int(tmax/dt))
y = np.empty(int(tmax/dt))
vx = np.empty(int(tmax/dt))
vy = np.empty(int(tmax/dt))

# Inicjalizacja tablic
x[0] = x0
y[0] = y0
vx[0] = vx0
vy[0] = vy0

counter = 0
end = None
# Pętla symulacji
for i in range(1, len(t)):
    r = radius(x[i-1], y[i-1])
    x[i] = x[i-1] + vx[i-1]*dt
    y[i] = y[i-1] + vy[i-1]*dt
    vx[i] = vx[i-1] - G*msol*x[i-1]*dt/(r**3)
    vy[i] = vy[i-1] - G*msol*y[i-1]*dt/(r**3)
    if x[i-1] <0 and x[i] > 0:
        counter+=1
    if counter == 3:
        end = i
        break

In [8]:
# Tworzenie wykresów 
fig, axes = plt.subplots(1, 2, figsize = (10, 10))
axes[0].plot(x[:end], y[:end])
axes[0].set_xlabel('x [m]')
axes[0].set_ylabel('y [m]')
axes[1].plot(t[:end], y[:end])
axes[1].set_xlabel('czas [s]')
axes[1].set_ylabel('y [m]')


# Zapis do pliku
if not os.path.exists(folderpath):
    os.mkdir(folderpath)
plt.savefig(folderpath + '/fig1.png')

In [4]:
# Warunki symulacji metodą RK4
dt = 15*60                                      #15 minut
tmax = int(3.75*76*365*24*60*60)                #3 okresy
time_interval = int(tmax/dt)
t = np.arange(0, tmax, dt, dtype = float)
x = np.empty(int(tmax/dt))
y = np.empty(int(tmax/dt))
vx = np.empty(int(tmax/dt))
vy = np.empty(int(tmax/dt))

# Inicjalizacja tablic
x[0] = x0
y[0] = y0
vx[0] = vx0
vy[0] = vy0

counter = 0
end = None

# Pętla symulacji
for i in range(1, len(t)):

    k = getRK4k(x[i-1], y[i-1], vx[i-1], vy[i-1], dt)

    ukx = x[i-1]
    uky = y[i-1]
    ukvx = vx[i-1]
    ukvy = vy[i-1]

    x[i] = ukx + (dt/6)*(k['x'][0] + 2*k['x'][1] + 2*k['x'][2] + k['x'][3])
    y[i] = uky + (dt/6)*(k['y'][0] + 2*k['y'][1] + 2*k['y'][2] + k['y'][3])
    vx[i] = ukvx + (dt/6)*(k['vx'][0] + 2*k['vx'][1] + 2*k['vx'][2] + k['vx'][3])
    vy[i] = ukvy + (dt/6)*(k['vy'][0] + 2*k['vy'][1] + 2*k['vy'][2] + k['vy'][3])

    if x[i-1] <0 and x[i] > 0:
        counter+=1
    if counter == 3:
        end = i
        break
    

In [5]:
# Tworzenie wykresów 
fig, axes = plt.subplots(1, 2, figsize = (10, 10))
axes[0].plot(x[:end], y[:end])
axes[0].set_xlabel('x [m]')
axes[0].set_ylabel('y [m]')
axes[1].plot(t[:end], y[:end])
axes[1].set_xlabel('czas [s]')
axes[1].set_ylabel('y [m]')


# Zapis do pliku
if not os.path.exists(folderpath):
    os.mkdir(folderpath)
plt.savefig(folderpath + '/fig2.png')

In [6]:
# Warunki symulacji metodą jawnego schematu Eulera ze zmiennym krokiem czasowym (tolerancja 1000m)
dt = 15*60                                      #15 minut
tmax = int(3.75*76*365*24*60*60)                #3 okresy
time_interval = int(tmax/dt)
t = np.arange(0, tmax, dt, dtype = float)
x = np.empty(int(tmax/dt))
y = np.empty(int(tmax/dt))
vx = np.empty(int(tmax/dt))
vy = np.empty(int(tmax/dt))

# Inicjalizacja tablic
x[0] = x0
y[0] = y0
vx[0] = vx0
vy[0] = vy0

counter = 0
end = None
c = 0.9 # czynnik tłumiący
tol = 1000 # m

# Pętla symulacji
for i in range(1, len(t)):
    r = radius(x[i-1], y[i-1])
    e = tol+1 # pierwszy obrót pętli while zagwarantowany
    xHalves = 0
    yHalves = 0
    vxHalves = 0
    vyHalves = 0
    while abs(e) > tol:

        xEst = x[i-1] + vx[i-1]*dt

        xHalvesTmp = x[i-1] + vx[i-1]*dt/2
        vxHalvesTmp = vx[i-1] - G*msol*x[i-1]*dt/(2*r**3)
        xHalves = xHalvesTmp + vxHalvesTmp*dt/2

        yEst = y[i-1] + vy[i-1]*dt

        yHalvesTmp = y[i-1] + vy[i-1]*dt/2
        vyHalvesTmp = vy[i-1] - G*msol*y[i-1]*dt/(2*r**3)
        yHalves = yHalvesTmp + vyHalvesTmp*dt/2

        vxHalves = vxHalvesTmp - G*msol*xHalvesTmp*dt/(2*r**3)
        vxHalves = vyHalvesTmp - G*msol*yHalvesTmp*dt/(2*r**3)

        ex = estimateError(xHalves, xEst, 'euler')
        ey = estimateError(yHalves, yEst, 'euler')
        if ex > ey:
            e = ex
        else:
            e = ey
        dt = c*dt*np.sqrt(tol/e)
    x[i] = xHalves
    y[i] = yHalves
    vx[i] = vx[i-1] - G*msol*x[i-1]*dt/(r**3)
    vy[i] = vy[i-1] - G*msol*y[i-1]*dt/(r**3)
    
    if x[i-1] <0 and x[i] > 0:
        counter+=1
    if counter == 3:
        end = i
        break

In [7]:
# Tworzenie wykresów 
fig, axes = plt.subplots(1, 2, figsize = (10, 10))
axes[0].plot(x[:end], y[:end])
axes[0].set_xlabel('x [m]')
axes[0].set_ylabel('y [m]')
axes[1].plot(t[:end], y[:end])
axes[1].set_xlabel('czas [s]')
axes[1].set_ylabel('y [m]')


# Zapis do pliku
if not os.path.exists(folderpath):
    os.mkdir(folderpath)
plt.savefig(folderpath + '/fig3.png')

In [10]:
# Warunki symulacji metodą jawnego schematu Eulera ze zmiennym krokiem czasowym (tolerancja 100m)
dt = 15*60                                      #15 minut
tmax = int(3.75*76*365*24*60*60)                #3 okresy
time_interval = int(tmax/dt)
t = np.arange(0, tmax, dt, dtype = float)
x = np.empty(int(tmax/dt))
y = np.empty(int(tmax/dt))
vx = np.empty(int(tmax/dt))
vy = np.empty(int(tmax/dt))

# Inicjalizacja tablic
x[0] = x0
y[0] = y0
vx[0] = vx0
vy[0] = vy0

counter = 0
end = None
c = 0.9 # czynnik tłumiący
tol = 100 # m

# Pętla symulacji
for i in range(1, len(t)):
    r = radius(x[i-1], y[i-1])
    e = tol+1 # pierwszy obrót pętli while zagwarantowany
    while abs(e) > tol:

        xEst = x[i-1] + vx[i-1]*dt

        xHalvesTmp = x[i-1] + vx[i-1]*dt/2 # 2 kroki z krokiem dt/2
        vxHalvesTmp = vx[i-1] - G*msol*x[i-1]*dt/(2*r**3)
        xHalves = xHalvesTmp + vxHalvesTmp*dt/2

        yEst = y[i-1] + vy[i-1]*dt

        yHalvesTmp = y[i-1] + vy[i-1]*dt/2
        vyHalvesTmp = vy[i-1] - G*msol*y[i-1]*dt/(2*r**3)
        yHalves = yHalvesTmp + vyHalvesTmp*dt/2

        ex = estimateError(xHalves, xEst, 'euler')
        ey = estimateError(yHalves, yEst, 'euler')
        if ex > ey:
            e = ex
        else:
            e = ey
        dt = c*dt*np.sqrt(tol/e)
    x[i] = xHalves
    y[i] = yHalves
    vx[i] = vx[i-1] - G*msol*x[i-1]*dt/(r**3)
    vy[i] = vy[i-1] - G*msol*y[i-1]*dt/(r**3)
    
    if x[i-1] <0 and x[i] > 0:
        counter+=1
    if counter == 3:
        end = i
        break

In [11]:
# Tworzenie wykresów 
fig, axes = plt.subplots(1, 2, figsize = (10, 10))
axes[0].plot(x[:end], y[:end])
axes[0].set_xlabel('x [m]')
axes[0].set_ylabel('y [m]')
axes[1].plot(t[:end], y[:end])
axes[1].set_xlabel('czas [s]')
axes[1].set_ylabel('y [m]')


# Zapis do pliku
if not os.path.exists(folderpath):
    os.mkdir(folderpath)
plt.savefig(folderpath + '/fig4.png')

In [18]:
# Warunki symulacji metodą RK4 ze zmiennym krokiem czasowym (tolerancja 1000m)
dt = 15*60                                      #15 minut
tmax = int(3.75*76*365*24*60*60)                #3 okresy
time_interval = int(tmax/dt)
t = np.arange(0, tmax, dt, dtype = float)
x = np.empty(int(tmax/dt))
y = np.empty(int(tmax/dt))
vx = np.empty(int(tmax/dt))
vy = np.empty(int(tmax/dt))

# Inicjalizacja tablic
x[0] = x0
y[0] = y0
vx[0] = vx0
vy[0] = vy0

counter = 0
end = None
c = 0.9 # czynnik tłumiący
tol = 1000 # m

# Pętla symulacji
for i in range(1, len(t)):
    r = radius(x[i-1], y[i-1])
    e = tol+1 # pierwszy obrót pętli while zagwarantowany
    xHalves = 0
    yHalves = 0
    vxHalves = 0
    vyHalves = 0
    while abs(e) > tol:

        ukx = x[i-1]
        uky = y[i-1]
        ukvx = vx[i-1]
        ukvy = vy[i-1]

        # zwykłe rk4
        k = getRK4k(x[i-1], y[i-1], vx[i-1], vy[i-1], dt)

        xEst = ukx + (dt/6)*(k['x'][0] + 2*k['x'][1] + 2*k['x'][2] + k['x'][3])
        yEst = uky + (dt/6)*(k['y'][0] + 2*k['y'][1] + 2*k['y'][2] + k['y'][3])

        # pierwsze pół rk4
        kHalvesTmp = getRK4k(x[i-1], y[i-1], vx[i-1], vy[i-1], dt/2)

        xHalvesTmp = ukx + (dt/12)*(kHalvesTmp['x'][0] + 2*kHalvesTmp['x'][1] + 2*kHalvesTmp['x'][2] + kHalvesTmp['x'][3])
        vxHalvesTmp = ukvx + (dt/12)*(kHalvesTmp['vx'][0] + 2*kHalvesTmp['vx'][1] + 2*kHalvesTmp['vx'][2] + kHalvesTmp['vx'][3])
        yHalvesTmp = uky + (dt/12)*(kHalvesTmp['y'][0] + 2*kHalvesTmp['y'][1] + 2*kHalvesTmp['y'][2] + kHalvesTmp['y'][3])
        vyHalvesTmp = ukvy + (dt/12)*(kHalvesTmp['vy'][0] + 2*kHalvesTmp['vy'][1] + 2*kHalvesTmp['vy'][2] + kHalvesTmp['vy'][3])
        
        #drugie pół rk4
        kHalves = getRK4k(xHalvesTmp, yHalvesTmp, vxHalvesTmp, vyHalvesTmp, dt/2)

        xHalves = xHalvesTmp + (dt/12)*(kHalves['x'][0] + 2*kHalves['x'][1] + 2*kHalves['x'][2] + kHalves['x'][3])
        yHalves = yHalvesTmp + (dt/12)*(kHalves['y'][0] + 2*kHalves['y'][1] + 2*kHalves['y'][2] + kHalves['y'][3])
        vxHalves = vxHalvesTmp + (dt/12)*(kHalves['vx'][0] + 2*kHalves['vx'][1] + 2*kHalves['vx'][2] + kHalves['vx'][3])
        vyHalves = vyHalvesTmp + (dt/12)*(kHalves['vy'][0] + 2*kHalves['vy'][1] + 2*kHalves['vy'][2] + kHalves['vy'][3])

        ex = estimateError(xHalves, xEst, 'rk4')
        ey = estimateError(yHalves, yEst, 'rk4')
        if ex > ey:
            e = ex
        else:
            e = ey
        dt = c*dt*pow(tol/e, 1/5)
    x[i] = xHalves
    y[i] = yHalves
    vx[i] = vxHalves
    vy[i] = vyHalves
    
    if x[i-1] <0 and x[i] > 0:
        counter+=1
    if counter == 3:
        end = i
        break

In [19]:
# Tworzenie wykresów 
fig, axes = plt.subplots(1, 2, figsize = (10, 10))
axes[0].plot(x[:end], y[:end])
axes[0].set_xlabel('x [m]')
axes[0].set_ylabel('y [m]')
axes[1].plot(t[:end], y[:end])
axes[1].set_xlabel('czas [s]')
axes[1].set_ylabel('y [m]')


# Zapis do pliku
if not os.path.exists(folderpath):
    os.mkdir(folderpath)
plt.savefig(folderpath + '/fig5.png')

In [ ]:
# Warunki symulacji metodą RK4 ze zmiennym krokiem czasowym (tolerancja 100m)
dt = 15*60                                      #15 minut
tmax = int(3.75*76*365*24*60*60)                #3 okresy
time_interval = int(tmax/dt)
t = np.arange(0, tmax, dt, dtype = float)
x = np.empty(int(tmax/dt))
y = np.empty(int(tmax/dt))
vx = np.empty(int(tmax/dt))
vy = np.empty(int(tmax/dt))

# Inicjalizacja tablic
x[0] = x0
y[0] = y0
vx[0] = vx0
vy[0] = vy0

counter = 0
end = None
c = 0.9 # czynnik tłumiący
tol = 100 # m

# Pętla symulacji
for i in range(1, len(t)):
    r = radius(x[i-1], y[i-1])
    e = tol+1 # pierwszy obrót pętli while zagwarantowany
    xHalves = 0
    yHalves = 0
    vxHalves = 0
    vyHalves = 0
    while abs(e) > tol:

        ukx = x[i-1]
        uky = y[i-1]
        ukvx = vx[i-1]
        ukvy = vy[i-1]

        # zwykłe rk4
        k = getRK4k(x[i-1], y[i-1], vx[i-1], vy[i-1], dt)

        xEst = ukx + (dt/6)*(k['x'][0] + 2*k['x'][1] + 2*k['x'][2] + k['x'][3])
        yEst = uky + (dt/6)*(k['y'][0] + 2*k['y'][1] + 2*k['y'][2] + k['y'][3])

        # pierwsze pół rk4
        kHalvesTmp = getRK4k(x[i-1], y[i-1], vx[i-1], vy[i-1], dt/2)

        xHalvesTmp = ukx + (dt/12)*(kHalvesTmp['x'][0] + 2*kHalvesTmp['x'][1] + 2*kHalvesTmp['x'][2] + kHalvesTmp['x'][3])
        vxHalvesTmp = ukvx + (dt/12)*(kHalvesTmp['vx'][0] + 2*kHalvesTmp['vx'][1] + 2*kHalvesTmp['vx'][2] + kHalvesTmp['vx'][3])
        yHalvesTmp = uky + (dt/12)*(kHalvesTmp['y'][0] + 2*kHalvesTmp['y'][1] + 2*kHalvesTmp['y'][2] + kHalvesTmp['y'][3])
        vyHalvesTmp = ukvy + (dt/12)*(kHalvesTmp['vy'][0] + 2*kHalvesTmp['vy'][1] + 2*kHalvesTmp['vy'][2] + kHalvesTmp['vy'][3])
        
        #drugie pół rk4
        kHalves = getRK4k(xHalvesTmp, yHalvesTmp, vxHalvesTmp, vyHalvesTmp, dt/2)

        xHalves = xHalvesTmp + (dt/12)*(kHalves['x'][0] + 2*kHalves['x'][1] + 2*kHalves['x'][2] + kHalves['x'][3])
        yHalves = yHalvesTmp + (dt/12)*(kHalves['y'][0] + 2*kHalves['y'][1] + 2*kHalves['y'][2] + kHalves['y'][3])
        vxHalves = vxHalvesTmp + (dt/12)*(kHalves['vx'][0] + 2*kHalves['vx'][1] + 2*kHalves['vx'][2] + kHalves['vx'][3])
        vyHalves = vyHalvesTmp + (dt/12)*(kHalves['vy'][0] + 2*kHalves['vy'][1] + 2*kHalves['vy'][2] + kHalves['vy'][3])

        ex = estimateError(xHalves, xEst, 'rk4')
        ey = estimateError(yHalves, yEst, 'rk4')
        if ex > ey:
            e = ex
        else:
            e = ey
        dt = c*dt*pow(tol/e, 1/5)
    x[i] = xHalves
    y[i] = yHalves
    vx[i] = vxHalves
    vy[i] = vyHalves
    
    if x[i-1] <0 and x[i] > 0:
        counter+=1
    if counter == 3:
        end = i
        break

In [16]:
# Tworzenie wykresów 
fig, axes = plt.subplots(1, 2, figsize = (10, 10))
axes[0].plot(x[:end], y[:end])
axes[0].set_xlabel('x [m]')
axes[0].set_ylabel('y [m]')
axes[1].plot(t[:end], y[:end])
axes[1].set_xlabel('czas [s]')
axes[1].set_ylabel('y [m]')


# Zapis do pliku
if not os.path.exists(folderpath):
    os.mkdir(folderpath)
plt.savefig(folderpath + '/fig6.png')